In [ ]:
import time
import os
import torch
import pytorch_mask_rcnn as pmr
    
# ------------------ adjustable parameters ---------------------
use_cuda = True                               # choose to use GPU or not
visualize_eval = True                         # choose to visualize evaluation results or not
val_num_samples = 5                           # number of samples during test, betweem 1 to 1444
ckpt_path = '../checkpoint.pth'               # path where to save the checkpoint.pth
data_dir = 'E:/PyTorch/data/VOC2012'          # dataset directory
# ------------------ adjustable parameters ---------------------
classes = pmr.dataset.VOC_BBOX_LABEL_NAMES

device = torch.device('cuda' if torch.cuda.is_available() and use_cuda else 'cpu')
print('cuda: {}\nuse_cuda: {}\n{} GPU(s) available'.format(torch.cuda.is_available(), use_cuda, torch.cuda.device_count()))
print('\ndevice: {}'.format(device))

valset = pmr.VOCDataset(data_dir, 'train', False, device=device) # len=1444
indices = torch.randperm(len(valset)).tolist()
valset = torch.utils.data.Subset(valset, indices[:val_num_samples])

model = pmr.maskrcnn_resnet50(True, 21).to(device)

In [ ]:
if os.path.exists(ckpt_path):
    checkpoint = torch.load(ckpt_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    del checkpoint
    torch.cuda.empty_cache()

since = time.time()
# ------------------val---------------------
model.eval()
for image in valset:
    with torch.no_grad():
        result = model(image)
        
    if visualize_eval:
        print('  '.join(classes[l] for l in result['labels']))
        print('  '.join('{:.2f}'.format(p) for p in result['scores']))
        pmr.show(image, result)
# ------------------val---------------------
print('total time: {:.2f} s'.format(time.time() - since))